From [VEST 2020](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/K7760H) documentation:

"Election results and precinct shapefile from the NC State Board of Elections (https://www.ncsbe.gov/results-data)

Buncombe 681, Henderson CV, Wake 01-07A, 07-07A are missing from the 20201018 shapefile. They are added from the 20190827 shapefile.

North Carolina produces two sets of election results data. The precinct results are the unaltered results as initially reported by the counties. Many counties report early votes by vote center while provisional and other nonstandard ballots may be reported countywide. The precinct-sorted results are then produced within 30 days after the election. In the precinct-sorted data nearly all votes are assigned to precincts regardless of the manner by which the ballots were cast. However, North Carolina law requires the addition of statistical "noise" to the precinct-sorted data wherever any given vote by any specific voter may otherwise be deduced via cross referencing the various election-related data sets produced by the SBE.

For the 2020 general election 51 counties reported all votes by precinct in their initial precinct results. The precinct-sorted data set was used instead for the counties listed below.

Alleghany, Avery, Beaufort, Bertie*, Bladen, Buncombe, Cabarrus, Caldwell, Camden, Currituck, Dare, Davidson*, Davie, Duplin*, Durham*, Edgecombe, Guilford, Halifax*, Harnett, Haywood, Henderson, Hertford, Hyde, Johnston, Jones, Lee, Lincoln, Macon, Martin, Mecklenburg*, Moore, Nash, New Hanover*, Northampton*, Orange, Pasquotank, Pitt*, Polk, Richmond, Scotland, Stokes*, Surry*, Tyrrell*, Wake, Washington, Watauga, Wayne, Wilkes*, Yadkin

In counties marked by asterisk some votes were still reported by vote center or countywide in the precinct-sorted data. These were distributed by candidate to precincts based on the precinct-level reported vote. The precinct-sorted results were further adjusted to match the certified countywide totals based on the precinct-level vote by candidate."

Note that the RDH checked which counties contained key words that would indicate sorting was required and found that the 2022 list of counties that needed sorting was down to 46 from 51. The list mostly overlaps with the above with a few added and removed. For more information see code below. 

**2022 RDH Processing:** 

Absentee and voting center votes were allocated proportionally to precincts, by share of precinct-reported vote.

The precinct shapefile available [here](https://www.nconemap.gov/datasets/voting-precincts/explore?location=35.097107%2C-79.888900%2C7.41) was last updated in March of 2023 and therefore has precinct names missing and that do not match the November 2022 election results. After reaching out to the NCSBE, we received the following response which led to all but 5 precinct names matching between the two files:

*I’m not sure which file that site is displaying/making available for download. But I would suggest using this one: https://s3.amazonaws.com/dl.ncsbe.gov/PrecinctMaps/SBE_PRECINCTS_20220831.zip, as it is the data closest to the Nov 2022 election while also being before the election.*

*We provide shapefiles on our ftp site, which is linked to on our Voting Maps/Redistricting page: https://www.ncsbe.gov/results-data/voting-maps-redistricting*

# Load packages and data

In [1]:
import pandas as pd
import geopandas as gp
import os
from pdv_functions import *
pd.options.display.max_columns = 100
'''
Sources:
precinct shp: https://s3.amazonaws.com/dl.ncsbe.gov/PrecinctMaps/SBE_PRECINCTS_20220831.zip
precinct election results: https://www.ncsbe.gov/results-data/election-results/historical-election-results-data
'''

'\nSources:\nprecinct shp: https://s3.amazonaws.com/dl.ncsbe.gov/PrecinctMaps/SBE_PRECINCTS_20220831.zip\nprecinct election results: https://www.ncsbe.gov/results-data/election-results/historical-election-results-data\n'

In [2]:
#gdf = gp.read_file("./raw-from-source/Voting_Precincts/Voting_Precincts.shp")
gdf = gp.read_file("./raw-from-source/SBE_PRECINCTS_20220831/SBE_PRECINCTS_20220831.shp")
df = pd.read_table("./raw-from-source/results_pct_20221108 (1).zip", sep = "\t")
sorted_prec = pd.read_csv("./raw-from-source/sorted_precincts/AllCounties.txt", sep="\t")

print("# prec ids in gdf not in df: ",len(set((gdf.county_nam.str.upper()+gdf.prec_id.str.upper()))-set(df.County.str.upper()+df.Precinct.str.upper())))
print("# prec ids in df not in gdf: ", len(set(df.County.str.upper()+df.Precinct.str.upper())-set(gdf.county_nam.str.upper()+gdf.prec_id.str.upper())))
print("shape df: ", (df.County.str.upper()+df.Precinct.str.upper()).nunique(), "\nshape gdf: ", (gdf.county_nam.str.upper()+gdf.prec_id.str.upper()).nunique())

/Users/lilyfalk/anaconda3/envs/pdv_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# prec ids in gdf not in df:  0
# prec ids in df not in gdf:  325
shape df:  2977 
shape gdf:  2652


# Process unsorted election results

From Ballotpedia:
- To include: US Senate, US House, State Senate, State House, State Supreme Court 
    - 'US SENATE'
    - 'US HOUSE OF REPRESENTATIVES DISTRICT XX'
    - 'NC STATE SENATE DISTRICT XX'
    - 'NC HOUSE OF REPRESENTATIVES DISTRICT XXX'
    - 'NC SUPREME COURT ASSOCIATE JUSTICE SEAT XX'
    
- Not sure: Intermediate Appelate Courts
    - 'NC COURT OF APPEALS JUDGE SEAT XX'
- Not to include: School boards, Municipal government, local ballot measures

## Grab info for column dictionaries

In [3]:
#Set party col
potential_party = df['Choice Party']
party_dict = {'DEM':'D','LIB':'L','REP':'R','UNA':'U','GRE':'G', "na":"N"}
df["col_party"] = df.loc[df['Choice Party'].isin(party_dict.keys()), "Choice Party"].map(party_dict)
df.loc[df["col_party"].isna(), 'col_party'] = "N"


#Set last name abrv - will need to edit 
df["col_last_name"] = "na"
df.loc[df["Choice"].str.contains(". "), "col_last_name"] = df["Choice"].str.split(pat=" ").str[2].str.slice(stop=3).str.upper()

#Correcting for unique instances
df.loc[(df["Choice"]=='Ted Davis, Jr.')|(df["Choice"]=='Gettys Cohen, Jr.')|(df["Choice"]=='Paul Lowe, Jr.')|(df["Choice"]=='Howard Penny, Jr.'), "col_last_name"] = df["Choice"].str.split(pat=" ").str[1].str.slice(stop=3).str.upper()
df.loc[(df["Choice"]=='Philip E. (Phil) Berger')|(df["Choice"]=='Mary Price (Pricey) Harrison')|(df["Choice"]=='Susan Lee (Susie) Scott')|(df["Choice"]=='Milton F. (Toby) Fitch'), "col_last_name"] = df["Choice"].str.split(pat=" ").str[3].str.slice(stop=3).str.upper()
df.loc[df["Choice"]=="Michael Greer O'Shea", "col_last_name"] = "OSH"

df.loc[df["col_last_name"].isna(), "col_last_name"] = df["Choice"].str.split(pat=" ").str[1].str.slice(stop=3).str.upper()
df.loc[df["Choice"] == "Write-In (Miscellaneous)", "col_last_name"] = "OWR"


#Set contest
general_office_dict = {"US SENATE":"USS", "US HOUSE": "CON", "STATE SENATE":"SU", "NC HOUSE OF REPRESENTATIVES": "SL", 
                       "NC SUPREME COURT": "SSC", "NC COURT OF APPEALS JUDGE":"IA"}
df["col_office"]='na'
df.loc[(df["Contest Name"].str.contains("US SENATE")), "col_office"] = "US SENATE"
df.loc[(df["Contest Name"].str.contains("US HOUSE")), "col_office"] = "US HOUSE"
df.loc[(df["Contest Name"].str.contains("STATE SENATE")), "col_office"] = "STATE SENATE"
df.loc[(df["Contest Name"].str.contains("NC HOUSE OF REPRESENTATIVES")), "col_office"] = "NC HOUSE OF REPRESENTATIVES"
df.loc[(df["Contest Name"].str.contains("NC SUPREME COURT")), "col_office"] = "NC SUPREME COURT"
df.loc[(df["Contest Name"].str.contains("NC COURT OF APPEALS JUDGE")), "col_office"] = "NC COURT OF APPEALS JUDGE"
df["office_abr"] = df["col_office"].map(general_office_dict)


#Set districts
#Get CONG DIST
df["col_cong_dist"] = "na"
df.loc[df["Contest Name"].str.contains("US HOUSE"), "col_cong_dist"] = df["Contest Name"].str.split(pat=" ").str[-1]
#GET state sen dist
df["col_su_dist"] = "na"
df.loc[df["Contest Name"].str.contains("STATE SENATE"), "col_su_dist"] = df["Contest Name"].str.split(pat=" ").str[-1]
#GET state house dist
df["col_sl_dist"] = "na"
df.loc[df["Contest Name"].str.contains("NC HOUSE OF REPRESENTATIVES"), "col_sl_dist"] = df["Contest Name"].str.split(pat=" ").str[-1]
#GET ssc seat
df["col_ssc_seat"] = "na"
df.loc[df["Contest Name"].str.contains("NC SUPREME COURT"), "col_ssc_seat"] = df["Contest Name"].str.split(pat=" ").str[-1]
#GET court of appeals dist
df["col_ia_seat"] = "na"
df.loc[df["Contest Name"].str.contains("NC COURT OF APPEALS JUDGE"), "col_ia_seat"] = df["Contest Name"].str.split(pat=" ").str[-1]


#Create column names
df["full_col_names"] = "na"
#cong
df.loc[df["Contest Name"].str.contains("US HOUSE"), "full_col_names"] = "G" + df["office_abr"] + df['col_cong_dist'] + df['col_party'] + df['col_last_name']
#us sen
df.loc[df["Contest Name"].str.contains("US SENATE"), "full_col_names"] = "G22" + df["office_abr"] + df['col_party'] + df['col_last_name']
#state sen
df.loc[df["Contest Name"].str.contains("STATE SENATE"), "full_col_names"] = "G" + df["office_abr"] + df['col_su_dist'].str.zfill(2) + df['col_party'] + df['col_last_name']
#state house
df.loc[df["Contest Name"].str.contains("NC HOUSE OF REPRESENTATIVES"), "full_col_names"] = "G" + df["office_abr"] + df['col_sl_dist'].str.zfill(3) + df['col_party'] + df['col_last_name']
#state ssc
df.loc[df["Contest Name"].str.contains("NC SUPREME COURT"), "full_col_names"] = "G22" + df["office_abr"] + df["col_ssc_seat"].str.zfill(2) + df['col_last_name']
#IA court
df.loc[df["Contest Name"].str.contains("NC COURT OF APPEALS JUDGE"), "full_col_names"] = "G22" + df["office_abr"] + df["col_ia_seat"].str.zfill(2) + df['col_last_name']


#filter
df = df[~df["office_abr"].isna()]
#Make dict
unsorted_df_column_name_dict = pd.Series((df["Choice"]+", "+df["Contest Name"]).values, index=df["full_col_names"]).to_dict()

## Pivot

In [4]:
## PIVOT RESULTS
df_pivot = df.pivot_table(index = ['County','Precinct'],
                         columns = ['full_col_names'],
                        values = ['Total Votes'],
                         aggfunc = 'sum')


#Clean up the indices
df_pivot.reset_index(inplace = True,drop=False)
df_pivot[('County', 'County')] = df_pivot[('County', '')]
df_pivot[('Precinct', 'Precinct')] = df_pivot[('Precinct', '')]


#Rename the columns
df_pivot.columns = df_pivot.columns.map(pd.Series([col[1] for col in df_pivot.columns], index = [col for col in df_pivot.columns]).to_dict())
df_pivot = df_pivot.fillna(0)


df_pivot["UNIQUE_ID"] = df_pivot["County"] + "---" + df_pivot["Precinct"]

## Separate out counties to use sorted results

In [5]:
searchfor = ['ABS', 'PROVISIONAL','PROVISIOINAL','PROVI ', 'PROV',
             'ONE STOP','ONESTOP','OS ','OS-',' OS','OSAP','OSCA',
             'OSCH','OSKD','OSLL','OSLOB','OSNR','OSOP','OSTA','OSWA',
             'OSDU','-OS','OSAV','OSBOE','OSGR','OSHS','OSJB','OSSE','OSWD',
             'OSCS','OSHT','MAOS','DBOS',
             'CURBSIDE','TRANS','LEE COUNTY BOE', 'MCSWAIN CENTER' 
            ]
in_sos =  df_pivot[df_pivot["Precinct"].str.contains('|'.join(searchfor))]
in_sos = in_sos.groupby(by=["County"]).sum().reset_index()
in_sos

,County,G22IA08FLO,G22IA08THO,G22IA09SAL,G22IA09STR,G22IA10ADA,G22IA10TYS,G22IA11JAC,G22IA11STA,G22SSC03DIE,G22SSC03INM,G22SSC05ALL,G22SSC05ERV,G22USSDBEA,G22USSGHOH,G22USSLBRA,G22USSNLEW,G22USSNOWR,G22USSRBUD,GCON01DDAV,GCON01RSMI,GCON02DROS,GCON02RVIL,GCON03DGAS,GCON03RMUR,GCON04DFOU,GCON04RGEE,GCON05DPAR,GCON05RFOX,GCON06DMAN,GCON06LWAT,GCON06RCAS,GCON07DGRA,GCON07RROU,GCON08DHUF,GCON08RBIS,GCON09DCLA,GCON09RHUD,GCON10DGEN,GCON10NJIM,GCON10NOWR,GCON10RMCH,GCON11DBEA,GCON11LCOA,GCON11REDW,GCON12DADA,GCON12RLEE,GCON13DNIC,GCON13RHIN,GCON14DJAC,...,GSU22LUBI,GSU22RCOL,GSU23DMEY,GSU23RWOO,GSU24DGIB,GSU24RBRI,GSU25DEWI,GSU25RGAL,GSU26NOWR,GSU26NROB,GSU26RBER,GSU27DGAR,GSU27RSES,GSU28DROB,GSU28RSCH,GSU29DCRU,GSU29RCRA,GSU30DJOH,GSU30RJAR,GSU31RKRA,GSU32DLOW,GSU32RWAR,GSU33DHOR,GSU33RFOR,GSU34DSAN,GSU34RNEW,GSU35RJOH,GSU36RSET,GSU37RSAW,GSU38DMOH,GSU39DSAL,GSU39RROB,GSU40DWAD,GSU40RSHI,GSU41DMAR,GSU41RLEO,GSU42DHUN,GSU42RRUS,GSU43ROVE,GSU44RALE,GSU45RPRO,GSU46DMAR,GSU46RDAN,GSU47RHIS,GSU48DCAR,GSU48RMOF,GSU49DMAY,GSU49RAND,GSU50DMCC,GSU50RCOR
0,ALLEGHANY,2199.0,967.0,889.0,2283.0,949.0,2206.0,936.0,2204.0,2221.0,998.0,2193.0,1017.0,963.0,20.0,48.0,0.0,2.0,2211.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,920.0,2328.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2413.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AVERY,1533.0,759.0,731.0,1563.0,756.0,1531.0,750.0,1536.0,1531.0,767.0,1502.0,799.0,777.0,12.0,32.0,0.0,1.0,1500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,755.0,1559.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1663.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BEAUFORT,6158.0,3829.0,3656.0,6362.0,3793.0,6203.0,3779.0,6183.0,6289.0,3781.0,6186.0,3878.0,3897.0,55.0,134.0,2.0,1.0,6038.0,0.0,0.0,0.0,0.0,3768.0,6330.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BERTIE,1267.0,2073.0,2101.0,1292.0,2106.0,1275.0,2099.0,1275.0,1316.0,2093.0,1282.0,2124.0,2137.0,15.0,35.0,0.0,1.0,1257.0,2216.0,1218.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BLADEN,3508.0,3218.0,3148.0,3577.0,3156.0,3547.0,3143.0,3493.0,3545.0,3228.0,3567.0,3184.0,3248.0,48.0,79.0,0.0,2.0,3476.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3207.0,3621.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,BUNCOMBE,22790.0,46789.0,44971.0,24560.0,46556.0,22969.0,46563.0,22899.0,23425.0,46454.0,22945.0,46944.0,46494.0,596.0,692.0,6.0,22.0,22347.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46492.0,810.0,22753.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12271.0,7699.0,0.0,0.0,0.0,34541.0,15146.0,0.0,0.0
6,CABARRUS,18023.0,18115.0,17478.0,18712.0,18096.0,18044.0,17914.0,18162.0,18148.0,18137.0,18022.0,18247.0,18362.0,241.0,426.0,0.0,15.0,17471.0,0.0,0.0,0.0,0

# Process state-sorted election results

In [6]:
#cut out over/under votes
sorted_prec = sorted_prec[~sorted_prec['result_type_desc'].str.contains("ER VOTES")]
sorted_prec['result_type_desc'].value_counts()

<NORMAL>    622877
WRITE-IN    158201
Name: result_type_desc, dtype: int64

## Grab info for column dictionaries

In [7]:
#Set party col
potential_party = sorted_prec['candidate_party_lbl']
party_dict = {'DEM':'D','LIB':'L','REP':'R','UNA':'U','GRE':'G', "na":"N"}
sorted_prec["col_party"] = sorted_prec.loc[sorted_prec['candidate_party_lbl'].isin(party_dict.keys()), 'candidate_party_lbl'].map(party_dict)
sorted_prec.loc[sorted_prec["col_party"].isna(), 'col_party'] = "N"


#Set last name abrv - will need to edit 
sorted_prec["col_last_name"] = "na"
#General cases
sorted_prec.loc[sorted_prec["candidate_name"].str.contains(". "), "col_last_name"] = sorted_prec["candidate_name"].str.split(pat=" ").str[2].str.slice(stop=3).str.upper()
#Correcting for unique instances
sorted_prec.loc[(sorted_prec["candidate_name"]=='Ted Davis, Jr.')|(sorted_prec["candidate_name"]=='Gettys Cohen, Jr.')|(sorted_prec["candidate_name"]=='Paul Lowe, Jr.')|(sorted_prec["candidate_name"]=='Howard Penny, Jr.'), "col_last_name"] = sorted_prec["candidate_name"].str.split(pat=" ").str[1].str.slice(stop=3).str.upper()
sorted_prec.loc[(sorted_prec["candidate_name"]=='Philip E. (Phil) Berger')|(sorted_prec["candidate_name"]=='Mary Price (Pricey) Harrison')|(sorted_prec["candidate_name"]=='Susan Lee (Susie) Scott')|(sorted_prec["candidate_name"]=='Milton F. (Toby) Fitch'), "col_last_name"] = sorted_prec["candidate_name"].str.split(pat=" ").str[3].str.slice(stop=3).str.upper()
sorted_prec.loc[sorted_prec["candidate_name"]=="Michael Greer O'Shea", "col_last_name"] = "OSH"
sorted_prec.loc[sorted_prec["col_last_name"].isna(), "col_last_name"] = sorted_prec["candidate_name"].str.split(pat=" ").str[1].str.slice(stop=3).str.upper()
sorted_prec.loc[sorted_prec["candidate_name"] == "Write-In (Miscellaneous)", "col_last_name"] = "OWR"


#Set contest
general_office_dict = {"US SENATE":"USS", "US HOUSE": "CON", "STATE SENATE":"SU", "NC HOUSE OF REPRESENTATIVES": "SL", 
                       "NC SUPREME COURT": "SSC", "NC COURT OF APPEALS JUDGE":"IA"}
sorted_prec["col_office"]='na'
sorted_prec.loc[(sorted_prec["contest_title"].str.contains("US SENATE")), "col_office"] = "US SENATE"
sorted_prec.loc[(sorted_prec["contest_title"].str.contains("US HOUSE")), "col_office"] = "US HOUSE"
sorted_prec.loc[(sorted_prec["contest_title"].str.contains("STATE SENATE")), "col_office"] = "STATE SENATE"
sorted_prec.loc[(sorted_prec["contest_title"].str.contains("NC HOUSE OF REPRESENTATIVES")), "col_office"] = "NC HOUSE OF REPRESENTATIVES"
sorted_prec.loc[(sorted_prec["contest_title"].str.contains("NC SUPREME COURT")), "col_office"] = "NC SUPREME COURT"
sorted_prec.loc[(sorted_prec["contest_title"].str.contains("NC COURT OF APPEALS JUDGE")), "col_office"] = "NC COURT OF APPEALS JUDGE"
sorted_prec["office_abr"] = sorted_prec["col_office"].map(general_office_dict)


#Set districts
#Get CONG DIST
sorted_prec["col_cong_dist"] = "na"
sorted_prec.loc[sorted_prec["contest_title"].str.contains("US HOUSE"), "col_cong_dist"] = sorted_prec["contest_title"].str.split(pat=" ").str[-1]
#GET state sen dist
sorted_prec["col_su_dist"] = "na"
sorted_prec.loc[sorted_prec["contest_title"].str.contains("STATE SENATE"), "col_su_dist"] = sorted_prec["contest_title"].str.split(pat=" ").str[-1]
#GET state house dist
sorted_prec["col_sl_dist"] = "na"
sorted_prec.loc[sorted_prec["contest_title"].str.contains("NC HOUSE OF REPRESENTATIVES"), "col_sl_dist"] = sorted_prec["contest_title"].str.split(pat=" ").str[-1]
#GET ssc seat
sorted_prec["col_ssc_seat"] = "na"
sorted_prec.loc[sorted_prec["contest_title"].str.contains("NC SUPREME COURT"), "col_ssc_seat"] = sorted_prec["contest_title"].str.split(pat=" ").str[-1]
#GET court of appeals dist
sorted_prec["col_ia_seat"] = "na"
sorted_prec.loc[sorted_prec["contest_title"].str.contains("NC COURT OF APPEALS JUDGE"), "col_ia_seat"] = sorted_prec["contest_title"].str.split(pat=" ").str[-1]


#Create column names
sorted_prec["full_col_names"] = "na"
#cong
sorted_prec.loc[sorted_prec["contest_title"].str.contains("US HOUSE"), "full_col_names"] = "G" + sorted_prec["office_abr"] + sorted_prec['col_cong_dist'] + sorted_prec['col_party'] + sorted_prec['col_last_name']
#us sen
sorted_prec.loc[sorted_prec["contest_title"].str.contains("US SENATE"), "full_col_names"] = "G22" + sorted_prec["office_abr"] + sorted_prec['col_party'] + sorted_prec['col_last_name']
#state sen
sorted_prec.loc[sorted_prec["contest_title"].str.contains("STATE SENATE"), "full_col_names"] = "G" + sorted_prec["office_abr"] + sorted_prec['col_su_dist'].str.zfill(2) + sorted_prec['col_party'] + sorted_prec['col_last_name']
#state house
sorted_prec.loc[sorted_prec["contest_title"].str.contains("NC HOUSE OF REPRESENTATIVES"), "full_col_names"] = "G" + sorted_prec["office_abr"] + sorted_prec['col_sl_dist'].str.zfill(3) + sorted_prec['col_party'] + sorted_prec['col_last_name']
#state ssc
sorted_prec.loc[sorted_prec["contest_title"].str.contains("NC SUPREME COURT"), "full_col_names"] = "G22" + sorted_prec["office_abr"] + sorted_prec["col_ssc_seat"].str.zfill(2) + sorted_prec['col_last_name']
#IA court
sorted_prec.loc[sorted_prec["contest_title"].str.contains("NC COURT OF APPEALS JUDGE"), "full_col_names"] = "G22" + sorted_prec["office_abr"] + sorted_prec["col_ia_seat"].str.zfill(2) + sorted_prec['col_last_name']


#filter
sorted_prec = sorted_prec[~sorted_prec["office_abr"].isna()]
#Set column key dict
sorted_df_column_name_dict = pd.Series((sorted_prec["candidate_name"]+", "+sorted_prec["contest_title"]).values, index=sorted_prec["full_col_names"]).to_dict()
#set countyfp
sorted_prec["COUNTYFP"] = 2*sorted_prec["county_id"]-1

## Pivot and filter for sorted counties

In [8]:
## PIVOT RESULTS
sorted_prec_pivot = sorted_prec.pivot_table(index = ['county','COUNTYFP','precinct_name'],
                         columns = ['full_col_names'],
                        values = ['vote_ct'],
                         aggfunc = 'sum')


#Clean up the indices
sorted_prec_pivot.reset_index(inplace = True,drop=False)
sorted_prec_pivot[('county', 'county')] = sorted_prec_pivot[('county', '')]
sorted_prec_pivot[('COUNTYFP', 'COUNTYFP')] = sorted_prec_pivot[('COUNTYFP', '')]
sorted_prec_pivot[('precinct_name', 'precinct_name')] = sorted_prec_pivot[('precinct_name', '')]


#Rename the columns
sorted_prec_pivot.columns = sorted_prec_pivot.columns.map(pd.Series([col[1] for col in sorted_prec_pivot.columns], index = [col for col in sorted_prec_pivot.columns]).to_dict())
sorted_prec_pivot = sorted_prec_pivot.fillna(0)
sorted_prec_pivot.drop([''], axis=1, inplace=True)

#separate out counties sorted by the state
sorted_counties = set(in_sos["County"].values)
df_sorted = sorted_prec_pivot[sorted_prec_pivot['county'].isin(sorted_counties)]
gdf_sorted = gdf[gdf['county_nam'].isin(sorted_counties)]

df_sorted["precinct_name"] = df_sorted["precinct_name"].str.strip() 
df_sorted["county"] = df_sorted["county"].str.strip()
df_sorted["UNIQUE_ID"] = df_sorted["county"].astype(str) + "---" + df_sorted["precinct_name"].astype(str)


<ipython-input-8-d5e4b9d4d5d8>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sorted["precinct_name"] = df_sorted["precinct_name"].str.strip()
<ipython-input-8-d5e4b9d4d5d8>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sorted["county"] = df_sorted["county"].str.strip()
<ipython-input-8-d5e4b9d4d5d8>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

## Joining sorted county df and gdf precinct

In [9]:
#Testing which precinct identifier column better matches
enr_merge = pd.merge(gdf_sorted, df_sorted.fillna(value=0), left_on=['county_nam','enr_desc'], right_on=['county','precinct_name'], how='outer', indicator=True)
prec_id_merge = pd.merge(gdf_sorted, df_sorted.fillna(value=0), left_on=['county_nam','prec_id'], right_on=['county','precinct_name'], how='outer', indicator=True)
#enr_merge[["county_nam", "enr_desc","prec_id", "county","precinct_name","_merge"]][enr_merge['_merge']!="both"].to_csv("./enr_merge.csv")
#prec_id_merge[["county_nam", "enr_desc","prec_id", "county","precinct_name","_merge"]][prec_id_merge['_merge']!="both"].to_csv("./prec_id_merge.csv")


#Correct precinct names to make them match between sorted df for isolated counties and gdf for isolated counties
gdf_sorted["prec"] = gdf_sorted["enr_desc"].str.upper()
gdf_sorted.loc[(gdf_sorted["county_nam"]=="BUNCOMBE")|(gdf_sorted["county_nam"]=="DURHAM"),"prec"] = gdf_sorted["prec_id"]
gdf_sorted.loc[(gdf_sorted["county_nam"]=="NORTHAMPTON")&(gdf_sorted["enr_desc"]=="GARYSBURG/PLEASA_PLEASANT HILL"),"prec"] = "GARYSBURG/PLEASANT HILL"
gdf_sorted.loc[(gdf_sorted["county_nam"]=="ORANGE")&(gdf_sorted["enr_desc"]=="HillsboroughEast"),"prec"] = "HILLSBOROUGH EAST"
gdf_sorted.loc[gdf_sorted["county_nam"]=="MECKLENBURG", "prec"] = "PCT "+gdf_sorted["enr_desc"].str.zfill(3)
gdf_sorted.loc[(gdf_sorted["county_nam"]=="MECKLENBURG")&(gdf_sorted["enr_desc"].str.endswith(".1")), "prec"] = "PCT "+gdf_sorted["enr_desc"].str.slice(stop=-2)
gdf_sorted.loc[(gdf_sorted["county_nam"]=="AVERY")&(gdf_sorted["prec_id"]=="14"), "prec"] = "NEWLAND 1"
gdf_sorted.loc[(gdf_sorted["county_nam"]=="AVERY")&(gdf_sorted["prec_id"]=="15"), "prec"] = "NEWLAND 2"
gdf_sorted.loc[(gdf_sorted["county_nam"]=="CASWELL")&(gdf_sorted["enr_desc"]=="YANCEYVILLE 2"), "prec"] = "YANCEYVILLE"
gdf_sorted.loc[(gdf_sorted["county_nam"]=="PITT")&(gdf_sorted["enr_desc"]=="GREENVILE  13A"), "prec"] = "GREENVILLE 13A"
gdf_sorted.loc[(gdf_sorted["county_nam"]=="JOHNSTON")&(gdf_sorted["prec_id"]=="PR25"), "prec"] = "WEST SELMA"
gdf_sorted.loc[gdf_sorted["county_nam"]=="WAKE", "prec"] = "PRECINCT "+gdf_sorted["enr_desc"].str.zfill(4)
gdf_sorted.loc[gdf_sorted["county_nam"]=="LEE", "prec"] = "PRECINCT "+gdf_sorted["enr_desc"]
gdf_sorted.loc[gdf_sorted["county_nam"]=="SCOTLAND", "prec"] = gdf_sorted["prec_id"].str.slice(stop=2)
df_sorted["prec"] = df_sorted["precinct_name"].str.upper()
df_sorted["prec"] = df_sorted["precinct_name"].str.replace("#","")
df_sorted["prec"] = df_sorted["prec"].astype(str).str.replace("  ", " ")
gdf_sorted["prec"] = gdf_sorted["prec"].astype(str).str.replace("  ", " ")


#Re-merge for max match
enr_prec_merge = pd.merge(gdf_sorted, df_sorted.fillna(value=0), left_on=['county_nam','prec'], right_on=['county','prec'], how='outer', indicator=True)
enr_prec_merge[["county_nam", "enr_desc","prec_id", "county","precinct_name","prec","_merge"]][enr_prec_merge["_merge"]!="both"].to_csv("./enr_prec_merge.csv")

/Users/lilyfalk/anaconda3/envs/pdv_env/lib/python3.9/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/lilyfalk/anaconda3/envs/pdv_env/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/Users/lilyfalk/anaconda3/envs/pdv_env/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A v

# Combine 

## Unsorted and sorted results

In [11]:
#Isolate counties from unsorted df that do not need sorting
df_pivot.drop([''], axis=1, inplace=True)
df_unsorted = df_pivot[~df_pivot['County'].isin(sorted_counties)]
df_sorted["County"] = df_sorted["county"]
#common columns are elction columns and UNIQUE_ID


df_sorted_no_dup_columns = df_unsorted[set(df_unsorted.columns)]
df_unsorted = df_unsorted[set(df_unsorted.columns)]
df_sorted = df_sorted[set(df_sorted.columns)]
#How is df[set(df.columns)] different from just df???

dff = pd.concat([df_unsorted, df_sorted], ignore_index=True)#, keys=['County', 'UNIQUE_ID'])
dff.shape

<ipython-input-11-89a150c49db0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sorted["County"] = df_sorted["county"]


(2655, 378)

In [12]:
dff['UNIQUE_ID'].nunique()

2655

## Join combined results with updated gdf

In [27]:
gdf_reform = gp.read_file("./raw-from-source/full_nc22_sn/full_nc22.shp")


#Correct precinct names to make them match between sorted df for isolated counties and gdf for isolated counties
gdf_reform["prec"] = gdf_reform["prec_id"].str.upper()
gdf_reform.loc[gdf_reform["county_nam"].isin(sorted_counties),"prec"] = gdf_reform["enr_desc"].str.upper()
gdf_reform.loc[(gdf_reform["county_nam"]=="BUNCOMBE")|(gdf_reform["county_nam"]=="DURHAM"),"prec"] = gdf_reform["prec_id"]
gdf_reform.loc[(gdf_reform["county_nam"]=="NORTHAMPTON")&(gdf_reform["enr_desc"]=="GARYSBURG/PLEASA_PLEASANT HILL"),"prec"] = "GARYSBURG/PLEASANT HILL"
gdf_reform.loc[(gdf_reform["county_nam"]=="ORANGE")&(gdf_reform["enr_desc"]=="HillsboroughEast"),"prec"] = "HILLSBOROUGH EAST"
gdf_reform.loc[gdf_reform["county_nam"]=="MECKLENBURG", "prec"] = "PCT "+gdf_reform["enr_desc"].str.zfill(3)
gdf_reform.loc[(gdf_reform["county_nam"]=="MECKLENBURG")&(gdf_reform["enr_desc"].str.endswith(".1")), "prec"] = "PCT "+gdf_reform["enr_desc"].str.slice(stop=-2)
gdf_reform.loc[(gdf_reform["county_nam"]=="AVERY")&(gdf_reform["prec_id"]=="14"), "prec"] = "NEWLAND 1"
gdf_reform.loc[(gdf_reform["county_nam"]=="AVERY")&(gdf_reform["prec_id"]=="15"), "prec"] = "NEWLAND 2"
gdf_reform.loc[(gdf_reform["county_nam"]=="CASWELL")&(gdf_reform["enr_desc"]=="YANCEYVILLE 2"), "prec"] = "YANCEYVILLE"
gdf_reform.loc[(gdf_reform["county_nam"]=="PITT")&(gdf_reform["enr_desc"]=="GREENVILE  13A"), "prec"] = "GREENVILLE 13A"
gdf_reform.loc[(gdf_reform["county_nam"]=="JOHNSTON")&(gdf_reform["prec_id"]=="PR25"), "prec"] = "WEST SELMA"
gdf_reform.loc[gdf_reform["county_nam"]=="WAKE", "prec"] = "PRECINCT "+gdf_reform["enr_desc"].str.zfill(4)
gdf_reform.loc[gdf_reform["county_nam"]=="LEE", "prec"] = "PRECINCT "+gdf_reform["enr_desc"]
gdf_reform.loc[gdf_reform["county_nam"]=="SCOTLAND", "prec"] = gdf_reform["prec_id"].str.slice(stop=2)
gdf_reform.loc[gdf_reform["enr_desc"]=="CV_CAROLINA VILLAGE", "prec"] = "CAROLINA VILLAGE"
df_sorted["prec"] = df_sorted["precinct_name"].str.upper()
df_sorted["prec"] = df_sorted["precinct_name"].str.replace("#","")
df_sorted["prec"] = df_sorted["prec"].astype(str).str.replace("  ", " ")
gdf_reform["prec"] = gdf_reform["prec"].astype(str).str.replace("  ", " ")

In [28]:
dff.loc[dff["precinct_name"].isna(),"prec"] = dff['Precinct']
dff['UNIQUE_ID'] = dff["County"] + "---" + dff["prec"]
gdf_reform['UNIQUE_ID'] = gdf_reform["county_nam"] + "---" + gdf_reform["prec"]
merge = pd.merge(gdf_reform, dff.fillna(value=0), on=['UNIQUE_ID'], how='outer', indicator=True)
merge[merge["_merge"]!="both"]

,DIS_COL,prec_id,enr_desc,county_nam,geometry,prec_x,UNIQUE_ID,GSL114DAGE,G22SSC05ALL,GSL104DLOF,GSU03RHAN,GSL033RDIN,GSU31RKRA,GSL116DRUD,GSL048DPIE,GSL066LPEN,GSL033DGIL,GCON04RGEE,GSU23RWOO,GSU28RSCH,GSU10DCOH,GSL028DMAY,GSL046RJON,GSU07RLEE,GCON01RSMI,GSU29RCRA,GSL036LWAR,GSU13DGRA,GSU24RBRI,GSU16DADC,GSU04DFIT,GSU16GTRU,GSL003RTYS,GSL089RSET,GSL067RSAS,GSL065RPYR,GCON08DHUF,GSL096RADA,GSL107RCOO,GSL088RPEA,GSL077RHOW,GSL034RSES,GSL051RSAU,GSL034DLON,GCON02RVIL,GSL059DYOU,G22USSNLEW,GSL100DAUT,GSL035DEVE,GSL109DHUG,...,GSL032RSOS,GSL098RBRA,GSL018DBUT,G22USSNOWR,GSL048RSWA,GSL009DFAR,GSU21RMCI,GCON11LCOA,GSL047DTOW,GSL107DALE,GSL075RLAM,GSL026DBEN,GSL029DALS,GSL083RCRU,GSL037DKEL,GCON06RCAS,GSL112DCOT,GSL118RPLE,GSU01RSAN,GSL015DSCH,GSU37RSAW,GSL025DGAI,GSL118DREM,GCON14RHAR,GSL022RBRI,GSL108RTOR,GSL025RCHE,GSL006RPIK,G22IA10ADA,GCON03RMUR,G22IA09STR,GSL050DPRI,GSL099DMAJ,GSL078DDAV,GSU42DHUN,GCON07NGRA,G22USSNBEA,COUNTYFP,GSL017NTER,GSL019NMIL,G22USSNHOH,GSL017NILE,precinct_name,G22USSNBRA,GCON07NROU,county,GSU08NRAB,G22USSNBUD,prec_y,_merge


In [ ]:
merge[merge.columns[merge.columns.str.contains("prec")]]

,prec_id,prec_x,precinct_name,prec_y
0,0001,0001,0,0001
1,0003,0003,0,0003
2,0003,0003,0,0003
3,0007,0007,0,0007
4,0008,0008,0,0008
...,...,...,...,...
2651,W,W,0,W
2652,W,W,0,W
2653,WW,WAYNESVILLE WEST,WAYNESVILLE WEST,WAYNESVILLE WEST
2654,YANC,YANCEYVILLE,YANCEYVILLE,YANCEYVILLE


In [29]:
merge[merge["prec_x"].isna()]

,DIS_COL,prec_id,enr_desc,county_nam,geometry,prec_x,UNIQUE_ID,GSL114DAGE,G22SSC05ALL,GSL104DLOF,GSU03RHAN,GSL033RDIN,GSU31RKRA,GSL116DRUD,GSL048DPIE,GSL066LPEN,GSL033DGIL,GCON04RGEE,GSU23RWOO,GSU28RSCH,GSU10DCOH,GSL028DMAY,GSL046RJON,GSU07RLEE,GCON01RSMI,GSU29RCRA,GSL036LWAR,GSU13DGRA,GSU24RBRI,GSU16DADC,GSU04DFIT,GSU16GTRU,GSL003RTYS,GSL089RSET,GSL067RSAS,GSL065RPYR,GCON08DHUF,GSL096RADA,GSL107RCOO,GSL088RPEA,GSL077RHOW,GSL034RSES,GSL051RSAU,GSL034DLON,GCON02RVIL,GSL059DYOU,G22USSNLEW,GSL100DAUT,GSL035DEVE,GSL109DHUG,...,GSL032RSOS,GSL098RBRA,GSL018DBUT,G22USSNOWR,GSL048RSWA,GSL009DFAR,GSU21RMCI,GCON11LCOA,GSL047DTOW,GSL107DALE,GSL075RLAM,GSL026DBEN,GSL029DALS,GSL083RCRU,GSL037DKEL,GCON06RCAS,GSL112DCOT,GSL118RPLE,GSU01RSAN,GSL015DSCH,GSU37RSAW,GSL025DGAI,GSL118DREM,GCON14RHAR,GSL022RBRI,GSL108RTOR,GSL025RCHE,GSL006RPIK,G22IA10ADA,GCON03RMUR,G22IA09STR,GSL050DPRI,GSL099DMAJ,GSL078DDAV,GSU42DHUN,GCON07NGRA,G22USSNBEA,COUNTYFP,GSL017NTER,GSL019NMIL,G22USSNHOH,GSL017NILE,precinct_name,G22USSNBRA,GCON07NROU,county,GSU08NRAB,G22USSNBUD,prec_y,_merge


In [ ]:
merge

## Format

In [30]:
merge['PRECINCT'] = merge['prec_x']
merge['COUNTYNM'] =  merge['county']
#Reorder
election_cols = merge.columns[merge.columns.str.startswith("G")].sort_values()
merge[election_cols] = merge[election_cols].astype(int)
#Sort alphabetically
merge = merge[['UNIQUE_ID','COUNTYFP', 'COUNTYNM','PRECINCT']+list(election_cols)]
merge

,UNIQUE_ID,COUNTYFP,COUNTYNM,PRECINCT,G22IA08FLO,G22IA08THO,G22IA09SAL,G22IA09STR,G22IA10ADA,G22IA10TYS,G22IA11JAC,G22IA11STA,G22SSC03DIE,G22SSC03INM,G22SSC05ALL,G22SSC05ERV,G22USSDBEA,G22USSGHOH,G22USSLBRA,G22USSNBEA,G22USSNBRA,G22USSNBUD,G22USSNHOH,G22USSNLEW,G22USSNOWR,G22USSRBUD,GCON01DDAV,GCON01RSMI,GCON02DROS,GCON02RVIL,GCON03DGAS,GCON03RMUR,GCON04DFOU,GCON04RGEE,GCON05DPAR,GCON05RFOX,GCON06DMAN,GCON06LWAT,GCON06RCAS,GCON07DGRA,GCON07NGRA,GCON07NROU,GCON07RROU,GCON08DHUF,GCON08RBIS,GCON09DCLA,GCON09RHUD,GCON10DGEN,GCON10NJIM,GCON10NOWR,...,GSU22LUBI,GSU22RCOL,GSU23DMEY,GSU23RWOO,GSU24DGIB,GSU24RBRI,GSU25DEWI,GSU25RGAL,GSU26NOWR,GSU26NROB,GSU26RBER,GSU27DGAR,GSU27RSES,GSU28DROB,GSU28RSCH,GSU29DCRU,GSU29RCRA,GSU30DJOH,GSU30RJAR,GSU31RKRA,GSU32DLOW,GSU32RWAR,GSU33DHOR,GSU33RFOR,GSU34DSAN,GSU34RNEW,GSU35RJOH,GSU36RSET,GSU37RSAW,GSU38DMOH,GSU39DSAL,GSU39RROB,GSU40DWAD,GSU40RSHI,GSU41DMAR,GSU41RLEO,GSU42DHUN,GSU42RRUS,GSU43ROVE,GSU44RALE,GSU45RPRO,GSU46DMAR,GSU46RDAN,GSU47RHIS,GSU48DCAR,GSU48RMOF,GSU49DMAY,GSU49RAND,GSU50DMCC,GSU50RCOR
0,BURKE---0001,0.0,0,0001,1095,438,427,1111,435,1099,445,1093,1104,440,1034,514,437,14,21,0,0,0,0,0,0,1081,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,439,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,424,1121,0,0,0,0,0,0,0
1,STANLY---0003,0.0,0,0003,672,498,478,689,495,672,498,669,669,505,665,514,503,14,16,0,0,0,0,0,2,646,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,494,680,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,492,683,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,BURKE---0003,0.0,0,0003,491,135,129,499,131,496,132,492,496,131,469,162,129,4,8,0,0,0,0,0,0,492,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,134,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,126,502,0,0,0,0,0,0,0
3,STANLY---0007,0.0,0,0007,723,396,359,762,386,730,380,738,732,388,729,392,379,13,21,0,0,0,0,0,0,709,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,381,739,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,383,733,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,STANLY---0008,0.0,0,0008,82,483,479,88,483,84,485,82,84,482,83,485,478,5,11,0,0,0,0,0,0,79,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,479,89,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,483,84,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2650,ORANGE---WESTWOOD,135.0,ORANGE,WESTWOOD,116,1345,1302,157,1347,111,1351,109,112,1356,115,1349,1358,10,9,0,0,0,0,0,0,97,0,0,0,0,0,0,1353,113,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1353,108,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2651,ALEXANDER---W,0.0,0,W,928,209,195,942,207,930,200,936,936,210,918,226,207,12,14,0,0,0,0,0,2,920,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,206,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1003,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2652,LENOIR---W,0.0,0,W,551,91,66,582,85,560,82,562,563,87,558,92,93,8,7,0,0,0,0,0,0,544,0,0,0,0,76,577,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2653,HAYWOOD---WAYNESVILLE WEST,87.0,HAYWOOD,WAYNESVILLE WEST,510,393,376,527,386,514,384,520,528,380,512,396,379,10,26,0,0,0,0,0,0,504,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,388,520


# Check vote totals

In [48]:
for column in merge.columns[merge.columns.str.startswith("G")]:
    print(column,": ", merge[column].sum())

G22IA08FLO :  1956780
G22IA08THO :  1775919
G22IA09SAL :  1700623
G22IA09STR :  2029212
G22IA10ADA :  1762821
G22IA10TYS :  1967407
G22IA11JAC :  1756037
G22IA11STA :  1968706
G22SSC03DIE :  1966040
G22SSC03INM :  1786648
G22SSC05ALL :  1957732
G22SSC05ERV :  1792820
G22USSDBEA :  1784031
G22USSGHOH :  30362
G22USSLBRA :  51634
G22USSNBEA :  0
G22USSNBRA :  0
G22USSNBUD :  0
G22USSNHOH :  0
G22USSNLEW :  83
G22USSNOWR :  2431
G22USSRBUD :  1905585
GCON01DDAV :  134802
GCON01RSMI :  122755
GCON02DROS :  190722
GCON02RVIL :  104168
GCON03DGAS :  82407
GCON03RMUR :  166500
GCON04DFOU :  194866
GCON04RGEE :  96500
GCON05DPAR :  102329
GCON05RFOX :  175245
GCON06DMAN :  139586
GCON06LWAT :  2810
GCON06RCAS :  116735
GCON07DGRA :  120220
GCON07NGRA :  0
GCON07NROU :  0
GCON07RROU :  164048
GCON08DHUF :  79248
GCON08RBIS :  184007
GCON09DCLA :  101227
GCON09RHUD :  131461
GCON10DGEN :  73174
GCON10NJIM :  110
GCON10NOWR :  242
GCON10RMCH :  194679
GCON11DBEA :  144161
GCON11LCOA :  5558
GCON1

In [33]:
merge.groupby(["COUNTYFP"]).sum()

,G22IA08FLO,G22IA08THO,G22IA09SAL,G22IA09STR,G22IA10ADA,G22IA10TYS,G22IA11JAC,G22IA11STA,G22SSC03DIE,G22SSC03INM,G22SSC05ALL,G22SSC05ERV,G22USSDBEA,G22USSGHOH,G22USSLBRA,G22USSNBEA,G22USSNBRA,G22USSNBUD,G22USSNHOH,G22USSNLEW,G22USSNOWR,G22USSRBUD,GCON01DDAV,GCON01RSMI,GCON02DROS,GCON02RVIL,GCON03DGAS,GCON03RMUR,GCON04DFOU,GCON04RGEE,GCON05DPAR,GCON05RFOX,GCON06DMAN,GCON06LWAT,GCON06RCAS,GCON07DGRA,GCON07NGRA,GCON07NROU,GCON07RROU,GCON08DHUF,GCON08RBIS,GCON09DCLA,GCON09RHUD,GCON10DGEN,GCON10NJIM,GCON10NOWR,GCON10RMCH,GCON11DBEA,GCON11LCOA,GCON11REDW,...,GSU22LUBI,GSU22RCOL,GSU23DMEY,GSU23RWOO,GSU24DGIB,GSU24RBRI,GSU25DEWI,GSU25RGAL,GSU26NOWR,GSU26NROB,GSU26RBER,GSU27DGAR,GSU27RSES,GSU28DROB,GSU28RSCH,GSU29DCRU,GSU29RCRA,GSU30DJOH,GSU30RJAR,GSU31RKRA,GSU32DLOW,GSU32RWAR,GSU33DHOR,GSU33RFOR,GSU34DSAN,GSU34RNEW,GSU35RJOH,GSU36RSET,GSU37RSAW,GSU38DMOH,GSU39DSAL,GSU39RROB,GSU40DWAD,GSU40RSHI,GSU41DMAR,GSU41RLEO,GSU42DHUN,GSU42RRUS,GSU43ROVE,GSU44RALE,GSU45RPRO,GSU46DMAR,GSU46RDAN,GSU47RHIS,GSU48DCAR,GSU48RMOF,GSU49DMAY,GSU49RAND,GSU50DMCC,GSU50RCOR
COUNTYFP,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,957697,583959,560954,981059,578335,962026,576005,962107,963414,586006,958931,590267,584825,11487,22002,0,0,0,0,60,1010,939064,55313,54806,0,0,61406,123058,40797,56582,61618,59197,26060,750,37410,69334,0,0,109565,56188,120822,66735,79423,72078,110,233,190952,32367,1752,66659,...,0,0,5719,9360,9087,17786,28031,47355,2057,0,23729,0,0,0,0,14221,44093,0,0,33527,46986,32220,19058,52235,0,0,56039,12928,54434,0,0,0,0,0,0,0,0,0,48218,58525,44497,13171,32667,26879,5648,17554,0,0,19601,37287
5.0,3630,1278,1140,3779,1247,3643,1232,3646,3665,1322,3647,1332,1255,40,84,0,0,0,0,0,2,3643,0,0,0,0,0,0,0,0,1190,3841,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3969,0,0,0,0,0,0
11.0,5172,1575,1496,5256,1558,5179,1531,5187,5177,1593,5131,1664,1595,54,92,0,0,0,0,0,4,5088,0,0,0,0,0,0,0,0,1543,5253,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5493,0,0,0,0,0,0
13.0,12533,6194,5912,12879,6131,12617,6088,12594,12770,6101,12613,6249,6254,107,283,0,0,0,0,0,7,12338,0,0,0,0,6030,12902,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15.0,2801,3366,3441,2848,3443,2836,3435,2827,2896,3431,2854,3468,3493,35,59,0,0,0,0,0,1,2781,3605,2736,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17.0,6790,4773,4665,6916,4677,6857,4641,6801,6860,4804,6906,4720,4799,83,149,0,0,0,0,0,7,6745,0,0,0,0,0,0,0,0,0,0,0,0,0,4750,0,0,7003,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
21.0,44220,74713,71577,47328,74332,44510,74250,44499,45411,74145,44562,75014,73833,1423,1651,0,0,0,0,6,53,43221,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,73904,1994,44102,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,18315,15042,0,0,0,56355,29446,0,0
25.0,42619,32133,30747,44061,31999,42722,31625,43003,42970,32126,42772,32282,32363,646,1148,0,0,0,0,0,50,41433,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4898,12286,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,31033,40960,2462,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
27.0,21247,6386,6200,21481,6383,21274,6382,21229,21355,6407,20892,6893,6421,195,390,0,0,0,0,0,21,20885,0,0,0,0,0,0,0,0,5150,17788,0,0,0,0,0,0,0,0,0,0,0,1096,0,9,3727,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16818,0,0,5810,0,0,0,0,0,0


# Export precinct boundary election results 

In [ ]:
#Make floats int
final[list(set(final.columns)-{'UNIQUE_ID','PRECINCT','cty_file_number','COUNTYFP','COUNTYNM'})]=final[list(set(final.columns)-{'UNIQUE_ID','PRECINCT','cty_file_number','COUNTYFP','COUNTYNM'})].astype(int)
#Reorder
election_cols = final.columns[final.columns.str.startswith("G")].sort_values()
#Sort alphabetically
final = final[['UNIQUE_ID','COUNTYFP', 'COUNTYNM','PRECINCT']+list(election_cols)]
final